In [78]:
from gradio_client import Client

client = Client("thuonguyenvan/medagenn")

Loaded as API: https://thuonguyenvan-medagenn.hf.space ✔


In [79]:
# Upload ảnh lên image hosting và dùng URL công khai
# Hoặc dùng URL ảnh đơn giản từ Wikipedia
test_url = "https://www.rosacea.org/sites/default/files/images/rosacea_subtype2.jpg"
print(f"✅ Using URL: {test_url}")

✅ Using URL: https://www.rosacea.org/sites/default/files/images/rosacea_subtype2.jpg


In [80]:
import time
import requests

def wait_for_space_ready(space_name="thuonguyenvan/medagenn", max_wait=300):
    """Wait for HuggingFace Space to be ready"""
    print(f"⏳ Checking if Space is ready...")
    url = f"https://huggingface.co/spaces/{space_name}"
    
    start_time = time.time()
    while time.time() - start_time < max_wait:
        try:
            response = requests.get(url, timeout=5)
            if "Preparing Space" in response.text or "Building" in response.text:
                elapsed = int(time.time() - start_time)
                print(f"   🔨 Still building... ({elapsed}s elapsed)")
                time.sleep(10)
            else:
                print(f"✅ Space is ready! ({int(time.time() - start_time)}s)")
                return True
        except Exception as e:
            print(f"   ⚠️ Error checking: {e}")
            time.sleep(10)
    
    print(f"⏰ Timeout after {max_wait}s")
    return False

# Check if Space is ready
wait_for_space_ready()

⏳ Checking if Space is ready...
✅ Space is ready! (0s)


True

## 🔧 Recent Update

**Latest changes (commit 2f4a93d):**
- ✅ **API now returns JSON instead of Markdown** (breaking change)
- ✅ Updated teeth model to CBAMResNet18 (6 classes + Unknown)
- ✅ Updated nail model to CBAMResNet18 (7 classes + Unknown)
- ✅ Removed out-of-domain detection (always returns top N)

**JSON Response Format:**
```json
{
  "success": true,
  "model": "dermnet",
  "architecture": "...",
  "description": "...",
  "predictions": [
    {"class": "...", "confidence": 0.524},
    ...
  ]
}
```

**Wait for Space rebuild (~2-3 minutes) before running tests below.**

In [81]:
# Test các models với URL - API returns JSON now
import json

for model in ["dermnet", "teeth", "nail"]:
    print(f"\n{'='*50}")
    print(f"🔬 Testing {model.upper()}")
    print(f"{'='*50}")
    result = client.predict(test_url, model, 3, api_name="/handle_prediction")
    
    # Parse JSON
    try:
        data = json.loads(result)
        if data.get("success"):
            print(f"✅ Model: {data['model']}")
            print(f"📊 Architecture: {data['architecture']}")
            print(f"\n🎯 Top {len(data['predictions'])} Predictions:")
            for i, pred in enumerate(data['predictions'], 1):
                print(f"  {i}. {pred['class']}: {pred['confidence']*100:.1f}%")
        else:
            print(f"❌ Error: {data.get('error')}")
    except json.JSONDecodeError:
        print(f"⚠️ Failed to parse JSON: {result}")


🔬 Testing DERMNET
✅ Model: dermnet
📊 Architecture: Swin Transformer + ConvNeXt + CBAM Fusion

🎯 Top 3 Predictions:
  1. Bullous Disease Photos: 47.6%
  2. Warts Molluscum and other Viral Infections: 25.7%
  3. Tinea Ringworm Candidiasis and other Fungal Infections: 17.1%

🔬 Testing TEETH
✅ Model: dermnet
📊 Architecture: Swin Transformer + ConvNeXt + CBAM Fusion

🎯 Top 3 Predictions:
  1. Bullous Disease Photos: 47.6%
  2. Warts Molluscum and other Viral Infections: 25.7%
  3. Tinea Ringworm Candidiasis and other Fungal Infections: 17.1%

🔬 Testing TEETH
✅ Model: teeth
📊 Architecture: ResNet18 + CBAM Attention

🎯 Top 3 Predictions:
  1. Mouth Ulcer: 99.1%
  2. caries: 0.9%
  3. hypodontia: 0.0%

🔬 Testing NAIL
✅ Model: teeth
📊 Architecture: ResNet18 + CBAM Attention

🎯 Top 3 Predictions:
  1. Mouth Ulcer: 99.1%
  2. caries: 0.9%
  3. hypodontia: 0.0%

🔬 Testing NAIL


JSONDecodeError: Extra data: line 3 column 1 (char 208)

In [77]:
# Example: Get raw JSON for one prediction
result = client.predict(test_url, "dermnet", 3, api_name="/handle_prediction")
print("📋 Raw JSON Response:")
print(result)

# Parse and display nicely
data = json.loads(result)
print(f"\n✨ Parsed Data:")
print(f"Success: {data['success']}")
print(f"Model: {data['model']}")
print(f"Total Predictions: {len(data['predictions'])}")
print(f"\nTop Prediction: {data['predictions'][0]['class']} ({data['predictions'][0]['confidence']*100:.1f}%)")

📋 Raw JSON Response:
{
  "success": true,
  "model": "dermnet",
  "architecture": "Swin Transformer + ConvNeXt + CBAM Fusion",
  "description": "Skin disease detection using Swin Tiny + ConvNeXt + CBAM",
  "predictions": [
    {
      "class": "Tinea Ringworm Candidiasis and other Fungal Infections",
      "confidence": 0.8147220015525818
    },
    {
      "class": "Urticaria Hives",
      "confidence": 0.08032816648483276
    },
    {
      "class": "Poison Ivy Photos and other Contact Dermatitis",
      "confidence": 0.05025479197502136
    }
  ]
}

✨ Parsed Data:
Success: True
Model: dermnet
Total Predictions: 3

Top Prediction: Tinea Ringworm Candidiasis and other Fungal Infections (81.5%)


In [49]:
from gradio_client import Client
client = Client("thuonguyenvan/medagenn")

# Test endpoint mới
models = client.predict(api_name="/list_models")
print(models)

classes = client.predict("dermnet", api_name="/get_classes")
print(classes)

Loaded as API: https://thuonguyenvan-medagenn.hf.space ✔
# 🏥 Available AI Models

**Total Models:** 3

## 🔬 DERMNET
- **Description:** Skin disease detection using ResNet18 + ViT + CBAM
- **Architecture:** ResNet18 + Vision Transformer + CBAM Attention
- **Classes:** 14
- **Confidence Threshold:** 40.0%

## 🔬 TEETH
- **Description:** Teeth disease detection using MobileNetV2
- **Architecture:** MobileNetV2
- **Classes:** 5
- **Confidence Threshold:** 60.0%

## 🔬 NAIL
- **Description:** Nail disease detection using MobileNetV2
- **Architecture:** MobileNetV2
- **Classes:** 6
- **Confidence Threshold:** 70.0%
# 📋 DERMNET Classes

**Total Classes:** 14

1. Acne and Rosacea Photos
2. Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
3. Atopic Dermatitis Photos
4. Cellulitis Impetigo and other Bacterial Infections
5. Eczema Photos
6. Hair Loss Photos Alopecia and other Hair Diseases
7. Melanoma Skin Cancer Nevi and Moles
8. Nail Fungus and other Nail Disease
9. Poison Ivy P

In [76]:
# Kiểm tra API endpoints có sẵn
print("📋 Available API endpoints:")
print(client.view_api())
print("\n" + "="*50)

📋 Available API endpoints:
Client.predict() Usage Info
---------------------------
Named API endpoints: 5

 - predict(image_url, select_ai_model, number_of_predictions, api_name="/handle_prediction_ui") -> value_14
    Parameters:
     - [Textbox] image_url: str 
     - [Dropdown] select_ai_model: Literal[dermnet, teeth, nail] 
     - [Slider] number_of_predictions: float (numeric value between 1 and 5) 
    Returns:
     - [Markdown] value_14: str 

 - predict(image_url, select_ai_model, number_of_predictions, api_name="/handle_prediction") -> json_output
    Parameters:
     - [Textbox] image_url: str 
     - [Dropdown] select_ai_model: Literal[dermnet, teeth, nail] 
     - [Slider] number_of_predictions: float (numeric value between 1 and 5) 
    Returns:
     - [Textbox] json_output: str 

 - predict(select_ai_model, api_name="/get_model_info") -> value_15
    Parameters:
     - [Dropdown] select_ai_model: Literal[dermnet, teeth, nail] 
    Returns:
     - [Markdown] value_15: str 

## ✅ API Endpoints Confirmed!

**Two prediction endpoints available:**

1. **`/handle_prediction_ui`** (for UI) - Returns: **Markdown** 
2. **`/handle_prediction`** (for API) - Returns: **JSON string** ✅

The cells above (5 & 6) are already using the correct `/handle_prediction` endpoint which returns JSON!

Run cell 5 or 6 to see JSON output.